In [23]:
import requests
import xmltodict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import torch
import time
from tqdm import tqdm

In [2]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 4060
Using device: cuda


In [8]:
# Install necessary libraries if not already installed:
# pip install transformers
# pip install adapter-transformers

from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch  # Required for handling tensors

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
model = AutoAdapterModel.from_pretrained('allenai/specter2_base')

# Load the adapter for Specter2
model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)


Fetching 4 files: 100%|██████████| 4/4 [00:00<?, ?it/s]
c:\Users\pilug\Documents\PaperSemanticSearch\.venv\lib\site-packages\adapters\loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

'specter2'

In [29]:
df = pd.read_json('papers.json', orient='records', lines=True)
df = df.drop_duplicates(subset=['title'], keep='first')
df = df[df['summary'].str.strip() != '']
df = df.reset_index(drop=True)
len(df)

2664

In [25]:
from transformers import AutoTokenizer
import torch


# Function to handle long texts with chunking
def embed_long_text(title, abstract, tokenizer, model, max_length=512):
    chunk_size = max_length - 10 
    chunks = [abstract[i:i + chunk_size] for i in range(0, len(abstract), chunk_size)]

    # Create tokenized inputs for each chunk
    embeddings = []
    for chunk in chunks:
        text = title + tokenizer.sep_token + chunk
        inputs = tokenizer(
            text,
            padding=True,
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=False,
            max_length=max_length
        )
        with torch.no_grad():  # Avoid gradient calculations for inference
            output = model(**inputs)
        # Take the first token in the chunk as the embedding
        embeddings.append(output.last_hidden_state[:, 0, :])

    # Aggregate embeddings (e.g., average)
    aggregated_embedding = torch.mean(torch.stack(embeddings), dim=0)
    return aggregated_embedding

batches = 50
batch_embeddings = []

embedding_file = "embeddings.pt"
if not os.path.exists(embedding_file) or os.path.getsize(embedding_file) == 0:
    torch.save([], embedding_file)  # Save an empty list initially

for idx, paper in tqdm(df.iterrows(), desc='Embedding Papers'):
    embedding = embed_long_text(
        paper['title'],
        paper['summary'],
        tokenizer,
        model
    )
    batch_embeddings.append(embedding)
    if len(batch_embeddings) == batches:
        existing_embeddings = torch.load(embedding_file)
        existing_embeddings.extend(batch_embeddings)
        print(f"Processed {idx} and {len(existing_embeddings)} embeddings")
        torch.save(existing_embeddings, embedding_file)
        batch_embeddings = []

# Save any remaining embeddings in the batch
if batch_embeddings:
    existing_embeddings = torch.load(embedding_file)
    existing_embeddings.extend(batch_embeddings)
    torch.save(existing_embeddings, embedding_file)
    print(f"Final total embeddings saved: {len(existing_embeddings)}")

Embedding Papers: 49it [00:15,  2.77it/s]C:\Users\pilug\AppData\Local\Temp\ipykernel_2092\2511239699.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  existing_embeddings

Final total embeddings saved: 2755
